**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install umap-learn
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from class_utils.plots import crosstab_plot

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import umap

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("UCI%20HAR%20Dataset.zip"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Znižovanie rozmeru dát na dátovej množine na rozpoznávanie ľudskej aktivity

V ďalšom príklade použijeme znižovanie rozmeru dát na [dátovú množinu o rozpoznávaní ľudských aktivít] (<https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones>). Dáta boli zozbierané pomocou smartfónových snímačov (akcelerometer, gyroskop) a transformované na množstvo rôznych sumárnych príznakov, ako sú priemer, smerodajná odchýlka, IRQ, energia, entropia atď.

Existuje 6 rôznych aktivít:

* chôdza;
* chôdza po poschodí;
* chôdza dole;
* sedenie;
* stojaci;
* kladenie.
### Načítavanie údajov

Údaje sú sériou čísel oddelených medzerami – tu je začiatok prvého riadku.



In [ ]:
with open("data/UCI HAR Dataset/train/X_train.txt", "r") as file:
    firstline = file.readline()
    print(firstline[:150], "...")

Dáta v tomto formáte je možné jednoducho načítať pomocou `np.loadtxt`. Pri načítavaní označení tried odpočítavame 1. Indexy označení začínajú od 1 a chceme, aby začínali od 0.



In [ ]:
X_train = np.loadtxt("data/UCI HAR Dataset/train/X_train.txt")
Y_train = np.loadtxt('data/UCI HAR Dataset/train/y_train.txt').astype(int) - 1

X_test = np.loadtxt("data/UCI HAR Dataset/test/X_test.txt")
Y_test = np.loadtxt('data/UCI HAR Dataset/test/y_test.txt').astype(int) - 1

Aby sme sa o dátach dozvedeli viac, môžeme sa pozrieť na README súbor.



In [ ]:
with open("data/UCI HAR Dataset/README.txt", 'r', errors="ignore") as file:
    print(file.read())

Jedna zaujímavá vec, ktorú README spomína, je, že príznaky sú už normalizované do rozsahu [-1, 1]. To je dobré, pretože nám to umožňuje znova preskočiť krok predspracovania.



In [ ]:
# WE DO NOT NEED THIS BECAUSE ACCORDING TO THE DOCS, THE DATA IS ALREADY
# NORMALIZED TO THE RANGE OF [-1, 1]

# input_preproc = make_pipeline(
#     SimpleImputer(),
#     StandardScaler()
# )

# X_train_preproc = input_preproc.fit_transform(X_train.reshape(X_train.shape[0], -1))
# X_train_preproc = X_train_preproc.reshape(X_train.shape)
# X_train = X_train_preproc

# X_test_preproc = input_preproc.transform(X_test.reshape(X_test.shape[0], -1))
# X_test_preproc = X_test_preproc.reshape(X_test.shape)
# X_test = X_test_preproc

Nakoniec získame zoznam názvov tried zo súboru, aby sme ho mohli použiť pri neskoršej analýze výsledkov.



In [ ]:
class_names = []

with open("data/UCI HAR Dataset/activity_labels.txt", 'r', errors="ignore") as file:
    for line in file:
        class_names.append(line[2:-1])

class_names = np.array(class_names)
print(class_names)

---
### Úloha 1: Aplikácia PCA na dátovú množinu

**V nasledujúcej bunke aplikujte na dátovú množinu metódu PCA (použite `X_train` a `Y_train`) a vykreslite výsledné body v 2D zafarbené podľa triedy. Názvy tried zobrazte v legende, aby sa dal graf ľahko interpretovať.** 

---


In [ ]:


# ----



---
### Úloha 2: Interpretácia PCA grafu

**Do nasledujúcej bunky vložte kvalitatívny opis toho, čo pozorujete v PCA grafe.** 

* Čo ste sa z grafu dozvedeli o štruktúre priestoru?
* Vychádzajúc z grafu, ktoré triedy si myslíte, že by bolo pre plytký klasifikátor ľahké správne oddeliť?
---


---
### Úloha 3: Aplikácia metódy UMAP na dátovú množinu

**V nasledujúcej bunke aplikujte na dátovú množinu metódu UMAP (použite `X_train` a `Y_train`) a vykreslite výsledné body v 2D zafarbené podľa triedy. Názvy tried zobrazte v legende, aby sa dal graf ľahko interpretovať.** 

---


In [ ]:
um = umap.UMAP(verbose=True)
points_umap = um.fit_transform(X_train)

perm_ind = np.random.permutation(points_umap.shape[0])
xx = points_umap[perm_ind]
yy = Y_train[perm_ind]
xt = X_train[perm_ind]

plt.figure(figsize=[10, 7])
cmap = plt.cm.get_cmap('jet', len(class_names))
plt.scatter(xx[:, 0], xx[:, 1], c=yy,
            cmap=cmap,
            rasterized=True)
cbar = plt.colorbar()
cbar.set_ticks(range(len(class_names)))
cbar.set_ticklabels(class_names)
plt.xlabel("dim 1")
plt.ylabel("dim 2")

---
### Úloha 4: Interpretácia UMAP grafu

**Do nasledujúcej bunky vložte kvalitatívny opis toho, čo pozorujete v UMAP grafe.** 

* Ako vyzerá štruktúra priestoru podľa tohto grafu?
* Vychádzajúc z grafu, ktoré triedy si myslíte, že by bolo pre plytký klasifikátor ľahké správne oddeliť?* Ako sa váš náhľad zmenil oproti tomu, čo ste sa dozvedeli z PCA grafu?
* Prečo je v tom rozdiel?

---


### Tréning jednoduchého klasifikátora

Ďalej sa si na dátovej množine natrénujeme jednoduchý klasifikátor, aby sme zistili, či sa potvrdí niektorá z intuícií, ktoré sme vyššie získali.



In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train);

Ďalej vyhodnotíme model na testovacej množine a najmä si vykreslíme maticu zámen. Mali by ste vidieť, že je ľahké odlíšiť aktivity, ktoré zahŕňajú veľa pohybu od aktivít, ktoré sú viacej statické.

V rámci týchto dvoch skupín sú výsledky viac zmiešané. Celkovo je však stále možné väčšinu vzoriek zatriediť správne aj s mimoriadne jednoduchým klasifikátorom bez akéhokoľvek ladenia hyperparametrov. Zdá sa, že trieda "laying" je obzvlášť ľahko rozpoznateľná.



In [ ]:
y_test = model.predict(X_test).astype(int)
acc = accuracy_score(Y_test, y_test)
print("Accuracy = {}".format(acc))

In [ ]:
# build a dataframe with Y_test and y_test
df = pd.DataFrame({"Y_test": Y_test, "y_test": y_test})
crosstab_plot("y_test", "Y_test", data=df)
plt.gca().set_xticklabels(class_names);
plt.gca().set_yticklabels(reversed(class_names));
plt.xlabel("Predicted")
plt.ylabel("Actual")